# Query the ADAPT NGA ESRI GeoDataBase
return a subset geopandas dataframe of the footprints of interest

In [11]:
import pandas as pd
import geopandas as gpd
import os
import maplib
import fiona

In [12]:
!pip install folium

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import folium
from folium import Map, TileLayer, GeoJson, LayerControl, Icon, Marker, features, Figure, CircleMarker, plugins
# This is for this:
# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.from_postgis.html
#from sqlalchemy import create_engine  
#!pip install psycopg2

##### This function to query the postgres database using SQL is NOT used here. Included for reference only.

In [14]:
def query_db_catid_NEW(catID, prod_code='M1BS', out_dir='/att/nobackup/pmontesa', db_table='nga_footprint_master_V2'):
    '''Query and select scenes from latest database
    '''
    with psycopg2.connect(database="arcgis", user="pmontesa", password="UzkEnLNgNXWnDZGeCQOH", host="arcdb04", port="5432") as dbConnect:

        cur = dbConnect.cursor() # setup the cursor
        selquery =  "SELECT S_FILEPATH, SENSOR, CATALOG_ID, ACQ_TIME FROM %s WHERE CATALOG_ID = '%s' AND PROD_CODE = '%s'" %(db_table, catID, prod_code)
        #selquery =  "SELECT * FROM %s WHERE CATALOG_ID = '%s' AND PROD_CODE = '%s'" %(db_table, catID, prod_code)
        cur.execute(selquery)
        selected=cur.fetchall()

    return selected

#### Identify the ESRI Geodatabase of interest

In [15]:
nga_gdb_fn = '/css/nga/INDEX/current/nga_footprint.gdb'
fiona.listlayers(nga_gdb_fn)

['nga_footprint_archive_v2', 'nga_footprint_master_v2']

#### Read in 1 row to list the columns of the GDB
- build a list of columns that you'll want to ignore when you read in the entire database - to keep memory usage low

In [16]:
# a tmp file of 1 row to get the col names of the gdb
nga_gdb_tmp = gpd.read_file(nga_gdb_fn, rows=1, layer='nga_footprint_master_v2')
print(nga_gdb_tmp.columns)
keep_col_list = ['s_filename', 'catalog_id']
rm_col_list = nga_gdb_tmp.columns.difference(keep_col_list).tolist()

Index(['fpuid', 'strip_id', 'scene_id', 'status', 'catalog_id', 'order_id',
       'prod_code', 'country', 'spec_type', 'acq_time', 'cloudcover',
       'cent_lat', 'cent_long', 'columns', 'rows', 'bits_pixel', 'file_fmt',
       'off_nadir', 'sun_elev', 'prod_gsd', 'ref_height', 'xtrackva',
       's_filename', 's_filepath', 'sensor', 'bands', 'previewjpg',
       'previewurl', 'added_date', 'file_sz', 'avsunazim', 'avtargetaz',
       'stereopair', 'pairname', 'acq_date', 'acq_year', 'prod_short',
       'det_pitch', 'intrackva', 'acq_month', 'zone_id', 'source',
       'shape_Length', 'shape_Area', 'geometry'],
      dtype='object')


#### Read in the Geodatabase and return just the columns you specified above

In [17]:
%%time
nga_gdb = gpd.read_file(nga_gdb_fn, layer='nga_footprint_master_v2') #ignore_fields = rm_col_list, 
nga_gdb.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 23443929 entries, 0 to 23443928
Data columns (total 45 columns):
 #   Column        Dtype   
---  ------        -----   
 0   fpuid         object  
 1   strip_id      object  
 2   scene_id      object  
 3   status        object  
 4   catalog_id    object  
 5   order_id      object  
 6   prod_code     object  
 7   country       object  
 8   spec_type     object  
 9   acq_time      object  
 10  cloudcover    float64 
 11  cent_lat      float64 
 12  cent_long     float64 
 13  columns       int64   
 14  rows          int64   
 15  bits_pixel    int64   
 16  file_fmt      object  
 17  off_nadir     float64 
 18  sun_elev      float64 
 19  prod_gsd      float64 
 20  ref_height    float64 
 21  xtrackva      float64 
 22  s_filename    object  
 23  s_filepath    object  
 24  sensor        object  
 25  bands         int64   
 26  previewjpg    object  
 27  previewurl    object  
 28  added_date    object  
 29  file

In [18]:
nga_gdb.head()

,fpuid,strip_id,scene_id,status,catalog_id,order_id,prod_code,country,spec_type,acq_time,...,acq_year,prod_short,det_pitch,intrackva,acq_month,zone_id,source,shape_Length,shape_Area,geometry
0,{904AB3F8-DAE7-4F3F-A196-491D5CEB05CF},WV02_103001003E79DF00_P1BS_500287766120_01,WV02_20150305160216_103001003E79DF00_15MAR0516...,validated_v1_2,103001003E79DF00,500287766120_01_P007,P1BS,YY,Panchromatic,2015-03-05T16:02:16.248375+00:00,...,2015,1B,0.008000,23.9,3,7,legacy_existing,1.015624,0.050648,"MULTIPOLYGON (((-64.90280 63.37973, -64.53114 ..."
1,{50CD65E0-51F0-490F-A61D-723C37116CE1},WV01_1020010027C66D00_P1BS_504883736020_01,WV01_20131215034409_1020010027C66D00_13DEC1503...,validated_v1_2,1020010027C66D00,504883736020_01_P002,P1BS,MN,Panchromatic,2013-12-15T03:44:09.827479+00:00,...,2013,1B,0.008000,-10.8,12,3,pgc_ftp,0.674994,0.017258,"MULTIPOLYGON (((109.98248 44.83392, 110.25690 ..."
2,{572C6B32-E888-4C66-B0FD-3A83880D172D},GE01_1050410001C42200_M1BS_054330448050_01,GE01_20100505182056_1050410001C42200_10MAY0518...,validated_v1_2,1050410001C42200,054330448050_01_P004,M1BS,US,Multispectral,2010-05-05T18:20:56.679428+00:00,...,2010,1B,0.038448,-9.6,5,9,pgc_ftp,0.602676,0.022648,"MULTIPOLYGON (((-114.36230 33.25610, -114.3577..."
3,{C61B6BD5-D36D-47E3-AFA9-7332143B3264},WV03_1040010060213800_M1BS_504682594010_01,WV03_20200915031840_1040010060213800_20SEP1503...,validated_v1_2,1040010060213800,504682594010_01_P006,M1BS,ID,Multispectral,2020-09-15T03:18:40.246550+00:00,...,2020,1B,0.032000,14.9,9,3,digitalglobe_ftp,0.608326,0.022803,"MULTIPOLYGON (((110.60576 -7.02609, 110.44999 ..."
4,{A2FA7950-9F3D-45DE-B23C-C3B9FE0B6421},WV02_103001002D840500_P1BS_500086173090_04,WV02_20140225091209_103001002D840500_14FEB2509...,validated_v1_2,103001002D840500,500086173090_04_P005,P1BS,YY,Panchromatic,2014-02-25T09:12:09.182375+00:00,...,2014,1B,0.008000,-20.6,2,1,legacy_existing,0.634462,0.024752,"MULTIPOLYGON (((18.98157 -26.57878, 18.80295 -..."


#### Read in a list of scenes names that we want footprints for

In [22]:
# The csv filename (fn) of the input raster (ntf scenes) file names...
r_fn_csv_fn = '/att/nobackup/pmontesa/userfs02/projects/srlite/misc/potential_scenes_4_srlite.csv'

r_fn_df = pd.read_csv(r_fn_csv_fn)

filename_search_list = [os.path.basename(f) for f in list(set(r_fn_df.path))]
subset_gdf = nga_gdb[nga_gdb['s_filename'].isin(filename_search_list)]

m = maplib.MAP_LAYER_FOLIUM(LAYER = subset_gdf, lat_start = subset_gdf.geometry.centroid.y.mean(), lon_start = subset_gdf.geometry.centroid.x.mean() )
m

<ipython-input-22-eafa9a63cf1b>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = maplib.MAP_LAYER_FOLIUM(LAYER = subset_gdf, lat_start = subset_gdf.geometry.centroid.y.mean(), lon_start = subset_gdf.geometry.centroid.x.mean() )


#### Write subset data frame of footprints to geopackage

In [19]:
!pip install utm

Defaulting to user installation because normal site-packages is not writeable


In [20]:
import utm
for idx, row in subset_gdf.iterrows():

    c = row.geometry.centroid
    
    utm_x, utm_y, band, zone = utm.from_latlon(c.y, c.x) 
    
    if c.y > 0: # Northern zone
        epsg = 32600 + band
    else:
        epsg = 32700 + band
        
    subset_gdf.loc[idx,'utm_zone'] = int(band)
    subset_gdf.loc[idx,'epsg'] = int(epsg)

    

NameError: name 'subset_gdf' is not defined

In [ ]:
subset_gdf

In [310]:
subset_gdf.to_file(os.path.splitext(r_fn_csv_fn)[0] + '.gpkg', driver='GPKG', layer='name', crs=4326) 
subset_gdf.to_file(os.path.splitext(r_fn_csv_fn)[0] + '.shp', driver='ESRI Shapefile', layer='name', crs=4326) 

#### Set up multiple bbox's

In [48]:
#bbox_serc = tuple([-76, 38, -75.5, 38.1])

bbox_ykd = [-165, 60, -162, 65]
bbox_sib = tuple([100, 65, 110, 70])

bbox_serc = tuple([-76.5627, 38.8903, -76.5555, 38.8957])
bbox_howland = tuple([-68.748274, 45.197211, -68.738740, 45.204889])
bbox_whitesands = tuple([-106.344218, 32.916644, -106.336254, 32.922880])
bbox_laselva =tuple([-84.010332, 10.426897, -84.002446, 10.434296])
SITES_BBOX_LIST = [bbox_serc, bbox_howland, bbox_whitesands, bbox_laselva] #, bbox_ykd, bbox_sib]
SITES_NAME_LIST = ['SERC', 'Howland', 'White Sands', 'La Selva']#, 'YK Delta', 'Siberia']

SITES_DICT = dict(zip(SITES_NAME_LIST, SITES_BBOX_LIST))

#### Subset by bounding box

In [2]:
xmin, ymin, xmax, ymax = bbox_ykd
z = subset_gdf.cx[xmin:xmax, ymin:ymax]
m = maplib.MAP_LAYER_FOLIUM(LAYER = z, lat_start = z.geometry.centroid.y.mean(), lon_start = z.geometry.centroid.x.mean() )
m

NameError: name 'subset_gdf' is not defined

##### Query the geodatabase with a spatial index using multiple bbox's.
use a bulk query with an optional predicate; predicate = 'contains'
https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.sindex.html#geopandas-geoseries-sindex

In [51]:
%%time
from folium import plugins
import shapely.geometry
BBOX_SERIES = gpd.GeoSeries([shapely.geometry.box(*bbox) for bbox in SITES_BBOX_LIST])

nga_gdb.sindex.query_bulk(BBOX_SERIES)

# index 1 returns what you want
z = nga_gdb.loc[nga_gdb.sindex.query_bulk(BBOX_SERIES)[1]]

print(z.shape)
#print(nga_gdb.sindex.query_bulk(BBOX_SERIES)[1])

m = maplib.MAP_LAYER_FOLIUM(LAYER = z.sample(frac=1), lat_start = z.geometry.centroid.y.mean(), lon_start = z.geometry.centroid.x.mean() ).add_child(plugins.MeasureControl()).add_child(plugins.Fullscreen())
m


(617, 45)


<timed exec>:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



CPU times: user 296 ms, sys: 50 ms, total: 346 ms
Wall time: 341 ms


Get SERC nga data for EVHR

In [72]:
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
z.s_filepath.to_csv(f'/att/nobackup/pmontesa/userfs02/projects/srlite/misc/nga_gdb_bbox_scenes_{nowtime}.csv')
z.shape

(617, 45)

In [42]:
z.cx[-76.5627:-76.5555, 38.8903:38.8957].shape
z.cx[-76.5627:-76.5555, 38.8903:38.8957].groupby('sun_elev')['catalog_id'].agg(['count'])

,count
sun_elev,
25.6,2
25.8,4
25.9,3
26.5,1
26.6,3
...,...
69.0,6
69.3,3
69.5,2


##### Example: see a footprint of all scenes of a given CATID

In [ ]:
search_list = ["1050010016726200"]
nga_gdb[nga_gdb['catalog_id'].isin(search_list)].plot()

##### Example: another style of searching with a list and plotting the result

In [ ]:
search_list = ['GE01_20190528002848_1050010016726200_19MAY28002848-M1BS-503250033040_01_P003.ntf']
nga_gdb.query('s_filename in @search_list').plot()